In [1]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import math
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s


def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo

In [5]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_new.csv',sep=',')

def N_s(Rp,P,i):
    os.system('mkdir result/N_tot_2020/')
    N_s_temp=0
    with open('result/N_tot_2020/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(stellar_table)):
            a_temp=a(stellar_table['mass'][i],P)
            tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
            delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
            n_temp=n(stellar_table['t_obs'][i],P)
            x=[3,6,12]
            y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(stellar_table['t_obs'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def N_s2(Rp,P):
#    os.system('mkdir result/N_s/')
    N_s_temp=0
#    with open('result/N_s/'+str(kepid)+'.csv','w') as f:
#        s=['SNR',
#           'fn',
#           'feff',]
#        writer=csv.DictWriter(f,fieldnames=s)
#        writer.writeheader()        
    for i in range(0,len(stellar_table)):
        a_temp=a(stellar_table['mass'][i],P)
        tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
        delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
        n_temp=n(stellar_table['t_obs'][i],P)
        x=[3,6,12]
        y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
        popt, pcov = curve_fit(sigma_s, x, y)
        sigma_LC=popt[0]
        cdppindex=popt[1]
        sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
        SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
        fn_temp=f_n(stellar_table['t_obs'][i],P)
        feff_temp=f_eff(SNR_temp)
#        writer.writerow({'SNR':SNR_temp,
#                         'fn':fn_temp,
#                         'feff':feff_temp,})                             
        N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

##############################################
def f_occurrence_cell_nofp(Rp_low,Rp_high,P_low,P_high): # the log base of Rp and P
    focc=pd.read_csv('result/occurrence_rate_tot_2020_2sigma.csv')
    f_occurrence_everyplanet=focc['occurrence_rate']
    f_id=focc['i']    
    count=0
    for i in range(0,len(f_occurrence_everyplanet)):
        if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high\
        and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(f_occurrence_everyplanet)):
            if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high\
            and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,f_occurrence_temp/count**(0.5)
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        Rs_temp=1.0 #unit R_sun G type star as sun
        Ms_temp=1.0 #unit M_sun
        a_temp=a(Ms_temp,P_center)
        e=0.1
        fgeo_temp=f_geo(Rp_center,Rs_temp,a_temp,e)
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,f_occurrence_temp
    
#####################################################3
def f_occurrence_cell_fp(Rp_low,Rp_high,P_low,P_high): # the log base of Rp and P
    focc=pd.read_csv('result/occurrence_rate_tot_2020_2sigma.csv')
    f_occurrence_everyplanet=focc['occurrence_rate']
    f_id=focc['i']    
    count=0
    for i in range(0,len(f_occurrence_everyplanet)):
        if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high:
#        and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(f_occurrence_everyplanet)):
            if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high:
#            and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,f_occurrence_temp/count**(0.5)
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        Rs_temp=1.0 #unit R_sun G type star as sun
        Ms_temp=1.0 #unit M_sun
        a_temp=a(Ms_temp,P_center)
        e=0.1
        fgeo_temp=f_geo(Rp_center,Rs_temp,a_temp,e)
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,f_occurrence_temp


In [3]:
for i in range(0,len(koi_table)):
    N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i],i)
    print(i)

KeyboardInterrupt: 

In [4]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_new.csv',sep=',')

import gc
f_occurrence_everyplanet=[]
#os.system('cd l')
from subprocess import getoutput
filelist=getoutput('ls result/N_tot_2020/*.csv')
filename=[]
for line in filelist.split('\n'):
    filename.append(line)
    
print(filename)
#os.system('cd ..')
with open('result/occurrence_rate_tot_2020_2sigma.csv','w+') as f:
    s=['i',
       'kepid',
       'occurrence_rate',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(filename)):
        koi_id=int(filename[i][18:-4])
        P=koi_table['koi_period'][koi_id]
        Rp=koi_table['koi_prad'][koi_id]
        Ms=koi_table['mass'][koi_id]
        Rs=koi_table['rad_b2018'][koi_id]
        a_temp=a(Ms,P)
        N_inf=pd.read_csv(filename[i],sep=',')
        fn=N_inf['fn']
        feff=N_inf['feff']
        N_s_temp=0
        for j in range(0,len(fn)):
            N_s_temp+=fn[j]*feff[j]
        
        del N_inf,fn,feff
        gc.collect()
        N_s_temp=round(N_s_temp)
        fgeo_temp=f_geo(Rp,Rs,a_temp,e)
        f_occurrence_temp=1./(fgeo_temp*N_s_temp)
        f_occurrence_everyplanet.append(f_occurrence_temp)
        print(koi_id,f_occurrence_temp,koi_table['kepid'][koi_id],len(koi_table))
        writer.writerow({'i':koi_id,
                         'kepid':koi_table['kepid'][koi_id],
                         'occurrence_rate':f_occurrence_temp,})
 

['result/N_tot_2020/0.csv', 'result/N_tot_2020/1000.csv', 'result/N_tot_2020/1001.csv', 'result/N_tot_2020/1002.csv', 'result/N_tot_2020/1003.csv', 'result/N_tot_2020/1004.csv', 'result/N_tot_2020/1005.csv', 'result/N_tot_2020/1006.csv', 'result/N_tot_2020/1007.csv', 'result/N_tot_2020/1008.csv', 'result/N_tot_2020/1009.csv', 'result/N_tot_2020/100.csv', 'result/N_tot_2020/1010.csv', 'result/N_tot_2020/1011.csv', 'result/N_tot_2020/1012.csv', 'result/N_tot_2020/1013.csv', 'result/N_tot_2020/1014.csv', 'result/N_tot_2020/1015.csv', 'result/N_tot_2020/1016.csv', 'result/N_tot_2020/1017.csv', 'result/N_tot_2020/1018.csv', 'result/N_tot_2020/1019.csv', 'result/N_tot_2020/101.csv', 'result/N_tot_2020/1020.csv', 'result/N_tot_2020/1021.csv', 'result/N_tot_2020/1022.csv', 'result/N_tot_2020/1023.csv', 'result/N_tot_2020/1024.csv', 'result/N_tot_2020/1025.csv', 'result/N_tot_2020/1026.csv', 'result/N_tot_2020/1027.csv', 'result/N_tot_2020/1028.csv', 'result/N_tot_2020/1029.csv', 'result/N_tot_

0 0.0014067407680984465 8182107 4713
1000 0.00016032798136932715 6962977 4713
1001 0.00023707445124659956 6962977 4713
1002 0.00034862301867038663 6962977 4713
1003 9.47037288118852e-05 6962977 4713
1004 0.00011736168999607165 6962977 4713
1005 0.00018293977029886825 6599919 4713
1006 0.00016262064918184184 4073089 4713
1007 8.659884027918957e-05 5466109 4713
1008 0.001253859416600612 11243547 4713
1009 0.000592395349082515 3340165 4713
100 3.4937651045921426e-05 12108364 4713
1010 0.00072287943511982 6599975 4713
1011 6.201858072612408e-05 10736179 4713
1012 0.00024086638694589935 9656252 4713
1013 7.52379445450307e-05 2707916 4713
1014 0.00025720707783811965 3340312 4713
1015 2.278846868858769e-05 2707985 4713
1016 0.00020831004994732364 3340313 4713
1017 0.00018692627860502576 5607631 4713
1018 1.1397796884800514e-05 10736223 4713
1019 0.0006470989908886195 4824268 4713
101 0.00033548640240768586 11757422 4713
1020 0.0001956710619153259 6675056 4713
1021 6.98816521547228e-05 2708172

1183 0.00018261441552417678 6541920 4713
1184 0.00026417548574437984 6541920 4713
1185 0.0008710120552013355 6541920 4713
1186 0.00029105243486500466 6933567 4713
1187 4.5980940196105815e-05 3836375 4713
1188 0.00028187824855 6863998 4713
1189 3.345477543551444e-05 5534476 4713
118 0.00019971278841008822 3854496 4713
1190 0.00019404143095172086 7175184 4713
1191 0.00012370439013475117 7175184 4713
1192 0.00017807346033441292 8547429 4713
1193 0.0008901562362060516 3230805 4713
1194 0.0004070247576526719 3230805 4713
1195 9.191535671278586e-05 8104030 4713
1196 0.000710950630713605 3656121 4713
1197 0.00035446672512862485 3656121 4713
1198 0.00010498272747347586 11410904 4713
1199 0.00033995887428748384 9909965 4713
119 7.524870331700565e-05 6525185 4713
11 5.562829833986007e-05 6628237 4713
1200 1.5927194766883365e-05 3836439 4713
1201 1.5927194766883365e-05 3836439 4713
1202 0.00025777413844214246 7094486 4713
1203 0.0004120421199336799 3836450 4713
1204 5.703242496037176e-05 3749365 

1366 9.536956623415898e-05 9965691 4713
1367 0.00012063765416211246 11650401 4713
1368 0.0003674671972797652 5535280 4713
1369 7.872379306820357e-05 8719419 4713
136 7.426362276930094e-05 5389956 4713
1370 0.15859732629468606 5097856 4713
1371 0.00025402564263291775 8950568 4713
1372 0.0005307456658169285 8950568 4713
1373 0.0012811501800105872 8950568 4713
1374 0.00019691694735325037 8950568 4713
1375 0.00013390694031844056 5097868 4713
1376 4.958252029948828e-05 5287983 4713
1377 0.0006406532156563799 2834805 4713
1378 0.00013072186050387685 9221398 4713
1379 0.0009090829080156197 3342467 4713
137 0.00011756119015950203 3554600 4713
1380 0.00254621689283104 3117046 4713
1381 5.811206320348625e-05 9180282 4713
1382 0.00012994156853930164 3438507 4713
1383 4.892636898688672e-05 8916492 4713
1384 0.0001697256241896556 8073806 4713
1385 0.00035781430468945875 4679314 4713
1386 0.0002595376040868951 7207061 4713
1387 0.0003023780150673367 7207061 4713
1388 0.00012921539055452103 10879038 

154 8.078824940280239e-05 12691412 4713
1550 0.00032664539608446446 11622985 4713
1551 0.0004730906891973413 6364276 4713
1552 0.00016250712514300508 6364294 4713
1553 4.422368899129404e-05 5480884 4713
1554 8.075317994324714e-05 6364307 4713
1555 0.0026375131852278398 5888187 4713
1556 5.3268524350808865e-05 2692377 4713
1557 3.73744847085123e-05 7552701 4713
1558 6.624527253638231e-05 7812167 4713
1559 2.0788802849899122e-05 8429415 4713
155 0.00020865180365612146 4633434 4713
1560 0.0001481366373623478 4544684 4713
1561 0.0005151676133418538 4644952 4713
1562 0.00014855281612417278 4644952 4713
1563 0.00012041783010899846 4644952 4713
1564 0.00016692326404058695 7812179 4713
1565 0.0018251867419364662 6364582 4713
1566 0.00019794266507411635 7203194 4713
1567 0.0001544385746290402 7977197 4713
1568 0.00022205103277394916 7977197 4713
1569 1.275748839056543e-05 8429450 4713
156 0.0003234155597135283 5526527 4713
1570 0.00011958693573963609 6364067 4713
1571 0.0008622350836340869 1038

...

4332 0.00011263348187174844 8344004 4713
4333 0.0005739748837595836 5632701 4713
4334 0.00131826944301293 6429812 4713
4335 0.0005957107600712029 10797460 4713
4336 0.00017770441258342746 10797460 4713
4337 0.0010616052603978276 8409295 4713
4338 0.00010953606377052253 5371776 4713
4339 0.00015667729139300942 5371776 4713
433 0.0003485134656565117 2142522 4713
4340 0.00023703385098775533 5371776 4713
4341 8.6907362339006e-05 5371776 4713
4342 3.34958673782282e-05 5371777 4713
4343 0.0005058215963727182 3966801 4713
4344 2.3745733375157823e-05 10847907 4713
4345 0.00013927252485755525 8216763 4713
4346 0.001793585013300783 8216763 4713
4347 0.0018854578292281706 8216763 4713
4348 4.3910437033944425e-05 6268722 4713
4349 0.00033546043214328344 3966912 4713
434 0.0006581232541033421 3758922 4713
4350 0.00030075246247695695 12208887 4713
4351 0.00038124068533807126 10074466 4713
4352 0.0002503990875797738 9969287 4713
4353 0.0007343489207346537 10982872 4713
4354 5.400392719378196e-05 1098

4515 0.00047241475579779544 9932970 4713
4516 0.0001380070379150511 6357290 4713
4517 9.194139331208427e-05 8628973 4713
4518 0.0004013418696423317 10141271 4713
4519 4.95242722925398e-05 7695093 4713
451 0.0004708873511365905 8951215 4713
4520 8.520502930438045e-05 11409698 4713
4521 0.00019712103021192525 10518725 4713
4522 0.0002009935413600088 5202905 4713
4523 0.0001578968591907508 5202905 4713
4524 0.00046110353467900813 7183745 4713
4525 0.00014097031140824408 7183745 4713
4526 0.00018811153687791444 9549471 4713
4527 0.0005723632861751477 7102227 4713
4528 6.0970654482431847e-05 7102227 4713
4529 0.0003107161210044996 7102227 4713
452 0.00039757056519554887 2994092 4713
4530 2.1168392678260393e-05 5041847 4713
4531 7.034422934347691e-05 11252617 4713
4532 4.348334474506401e-05 11252617 4713
4533 0.0015727146525547093 8210018 4713
4534 0.0003511741233032091 4650733 4713
4535 0.0014345825439476259 10907307 4713
4536 0.00010165952922746168 10583180 4713
4537 5.885770107512919e-05 

4699 0.00026852286737419454 10717241 4713
469 0.00019580390825911685 12016243 4713
46 0.000893282184865697 6941084 4713
4700 0.00034535916799504826 9540675 4713
4701 0.0008397758603035425 8212002 4713
4702 0.0022863723348422824 8212002 4713
4703 0.0002668246072954979 8396405 4713
4704 1.5815642264769353e-05 9222516 4713
4705 5.1372804727847126e-05 2696703 4713
4706 0.003971402491745585 9458266 4713
4707 1.729111209460123e-05 11724091 4713
4708 0.00017341033426241412 8106610 4713
4709 0.0002448649993609816 9850843 4713
470 6.610061832785141e-05 7778767 4713
4710 0.00012372705159543925 11724094 4713
4711 0.0005081533664155165 4946581 4713
4712 0.00015700798309672672 7090524 4713
471 0.00018598513291264666 7778767 4713
472 3.0573093861054874e-05 6294819 4713
473 0.0004749745257982548 7918172 4713
474 8.249480171112502e-05 9899514 4713
475 0.000248673198524353 3531558 4713
476 0.00020460706477390105 9899577 4713
477 7.324013161393708e-05 9899607 4713
478 0.0003229994767073296 9573685 4713


643 3.6018755081954576e-05 2989706 4713
644 0.0002917183995360314 7916058 4713
645 4.856588898296888e-05 11463950 4713
646 0.0001283630749768392 11457726 4713
647 0.0006346965271258065 9594184 4713
648 0.00015737363638256314 7501646 4713
649 3.345210765480597e-05 7032568 4713
64 0.005902557156833366 9950612 4713
650 0.00027519180809824656 9570741 4713
651 2.794614097206777e-05 7032687 4713
652 0.00020288665139548822 11656246 4713
653 3.475168329522465e-05 5802292 4713
654 3.8702410086441635e-05 7770450 4713
655 1.6670270779091032e-05 7770471 4713
656 0.00031139382602683133 9405595 4713
657 7.957073513859328e-05 3757407 4713
658 0.0001015539700355842 7289165 4713
659 0.00036073850246953136 11153476 4713
65 0.00039484331390775604 9950612 4713
660 3.818484587336704e-05 6778008 4713
661 4.863015848130325e-05 5802470 4713
662 4.320367126380565e-05 6778019 4713
663 9.197469349046305e-05 5802478 4713
664 9.385062939402341e-05 9898017 4713
665 8.98979049119312e-05 11153539 4713
666 0.000148372

830 0.0003346692727255622 6962109 4713
831 0.0011008491621254375 5900235 4713
832 0.00014476789055147975 9450647 4713
833 3.072528785625828e-05 5171707 4713
834 4.38817205908982e-05 9541094 4713
835 0.00038088896877242227 7025526 4713
836 0.00024598314927516105 5597361 4713
837 0.00028868213470074054 9636563 4713
838 0.005970324542238837 10844823 4713
839 0.0002060278134669571 2837111 4713
83 0.0004291958712512129 3554031 4713
840 0.00013834802738271637 2837111 4713
841 2.6860055537554563e-05 9541163 4713
842 3.246478283329088e-05 7211879 4713
843 0.00013725830763789345 11402995 4713
844 0.0012410302822913198 4680772 4713
845 0.0001745374457670819 6029239 4713
846 0.011560226138678208 6029239 4713
847 0.00013678455614144072 9934208 4713
848 8.238141221339471e-05 10318874 4713
849 3.582512572595839e-05 8189801 4713
84 0.00022666786449073654 3554031 4713
850 0.0001455925679865866 6345732 4713
851 0.00021667286836444491 6345732 4713
852 4.3663931960504244e-05 6595662 4713
853 0.0001069891

In [6]:
bins_period=np.logspace(math.log(0.4,5),math.log(250,5),13,base=5)
bins_radius=np.logspace(-1.5,5.5,15,base=2)   

occurrence_cell=np.zeros((14,12))
std_cell=np.zeros((14,12))
count_cell=np.zeros((14,12))
for i in range(0,len(bins_radius)-1):
    for j in range(0,len(bins_period)-1):
        Rp_low=bins_radius[i]
        Rp_high=bins_radius[i+1]
        P_low=bins_period[j]
        P_high=bins_period[j+1]
        temp=f_occurrence_cell_nofp(Rp_low,Rp_high,P_low,P_high)
        count_cell[i,j]=temp[0]
        occurrence_cell[i,j]=temp[1]
        std_cell[i,j]=temp[2]
        print(i,j)

soften_factor=1./np.log10(2)/np.log10(5)
print(soften_factor)
with open('result/occurrence_rate_2sigma_tot_cell_nofp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(bins_radius)-1):
        for j in range(0,len(bins_period)-1):
            writer.writerow({'i':i,
                             'j':j,
                             'count_cell':count_cell[i,j],
                             'occurrence_cell':occurrence_cell[i,j]*soften_factor,
                             'std_cell':std_cell[i,j]*soften_factor})

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
4.752604652960755


In [7]:
bins_period=np.logspace(math.log(0.4,5),math.log(250,5),13,base=5)
bins_radius=np.logspace(-1.5,5.5,15,base=2)   

occurrence_cell=np.zeros((14,12))
std_cell=np.zeros((14,12))
count_cell=np.zeros((14,12))
for i in range(0,len(bins_radius)-1):
    for j in range(0,len(bins_period)-1):
        Rp_low=bins_radius[i]
        Rp_high=bins_radius[i+1]
        P_low=bins_period[j]
        P_high=bins_period[j+1]
        temp=f_occurrence_cell_fp(Rp_low,Rp_high,P_low,P_high)
        count_cell[i,j]=temp[0]
        occurrence_cell[i,j]=temp[1]
        std_cell[i,j]=temp[2]
        print(i,j)

soften_factor=1./np.log10(2)/np.log10(5)
print(soften_factor)
with open('result/occurrence_rate_2sigma_tot_cell_fp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(bins_radius)-1):
        for j in range(0,len(bins_period)-1):
            writer.writerow({'i':i,
                             'j':j,
                             'count_cell':count_cell[i,j],
                             'occurrence_cell':occurrence_cell[i,j]*soften_factor,
                             'std_cell':std_cell[i,j]*soften_factor})

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
4.752604652960755


In [8]:
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_fp.csv")
soften_factor1=1./np.log10(bins_period[1]/bins_period[0])/np.log10(bins_radius[1]/bins_radius[0])
soften_factor2=1./np.log10(2)/np.log10(5)
factor=soften_factor1/soften_factor2
with open('result/occurrence_rate_2sigma_tot_cell_fp_cp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(data)):
        writer.writerow({'i':data['i'][i],
                         'j':data['j'][i],
                         'count_cell':data['count_cell'][i],
                         'occurrence_cell':data['occurrence_cell'][i]*factor,
                         'std_cell':data['std_cell'][i]*factor})

In [9]:
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_nofp.csv")
soften_factor1=1./np.log10(bins_period[1]/bins_period[0])/np.log10(bins_radius[1]/bins_radius[0])
soften_factor2=1./np.log10(2)/np.log10(5)
factor=soften_factor1/soften_factor2
with open('result/occurrence_rate_2sigma_tot_cell_nofp_cp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(data)):
        writer.writerow({'i':data['i'][i],
                         'j':data['j'][i],
                         'count_cell':data['count_cell'][i],
                         'occurrence_cell':data['occurrence_cell'][i]*factor,
                         'std_cell':data['std_cell'][i]*factor})

In [10]:
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_fp_cp.csv")

with open('result/occurrence_rate_2sigma_tot_cell_fp_new.csv','w') as f:
    s=['0','1','2','3','4','5','6','7','8','9','10','11']
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(27,-1,-1):
        temp=[]
        for j in range(0,12):         
            if np.mod(i,2)!=0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('')
                    
                else:
                    temp.append('{:.2g}'.format(data['occurrence_cell'][int(i/2)*12+j]))
                    
            
            elif np.mod(i,2)==0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('<'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j]))   
                                                 
                else:
                    temp.append('$\pm$'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j])) 
                    
                                        
        writer.writerow({'0':temp[0],
                         '1':temp[1],
                         '2':temp[2],
                         '3':temp[3],
                         '4':temp[4],
                         '5':temp[5],
                         '6':temp[6],
                         '7':temp[7],
                         '8':temp[8],
                         '9':temp[9],
                         '10':temp[10],
                         '11':temp[11],})
        
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_fp_new.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0028,0.015,0.011,0.019,0.038,0.044,0.062,0.076,0.13,0.092,0.11,0.25
1,$\pm$0.0011,$\pm$0.003,$\pm$0.003,$\pm$0.0046,$\pm$0.0084,$\pm$0.011,$\pm$0.015,$\pm$0.02,$\pm$0.032,$\pm$0.031,$\pm$0.042,$\pm$0.078
2,0.0094,0.017,0.027,0.038,0.047,0.056,0.073,0.11,0.09,0.085,0.038,0.052
3,$\pm$0.0023,$\pm$0.0035,$\pm$0.0053,$\pm$0.0076,$\pm$0.0098,$\pm$0.012,$\pm$0.017,$\pm$0.026,$\pm$0.03,$\pm$0.032,$\pm$0.027,$\pm$0.037
4,0.0084,0.017,0.006,0.023,0.025,0.061,0.059,0.074,0.073,0.02,0.014,0.027
5,$\pm$0.0023,$\pm$0.0037,$\pm$0.0027,$\pm$0.006,$\pm$0.007,$\pm$0.014,$\pm$0.016,$\pm$0.022,$\pm$0.028,$\pm$0.02,$\pm$0.014,$\pm$0.027
6,0.0024,0.0069,0.0094,0.031,0.024,0.025,0.028,0.052,0.084,0.085,0.15,0.03
7,$\pm$0.0012,$\pm$0.0024,$\pm$0.0031,$\pm$0.0074,$\pm$0.0071,$\pm$0.0083,$\pm$0.011,$\pm$0.017,$\pm$0.03,$\pm$0.035,$\pm$0.063,$\pm$0.03
8,0.0014,0.0035,0.0069,0.023,0.016,0.022,0.052,0.015,0.089,0.099,0.31,0.21
9,$\pm$0.00081,$\pm$0.0018,$\pm$0.0028,$\pm$0.006,$\pm$0.0059,$\pm$0.0082,$\pm$0.016,$\pm$0.011,$\pm$0.031,$\pm$0.04,$\pm$0.085,$\pm$0.084


In [12]:
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_nofp_cp.csv")

with open('result/occurrence_rate_2sigma_tot_cell_nofp_new.csv','w') as f:
    s=['0','1','2','3','4','5','6','7','8','9','10','11']
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(27,-1,-1):
        temp=[]
        for j in range(0,12):         
            if np.mod(i,2)!=0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('')
                    
                else:
                    temp.append('{:.2g}'.format(data['occurrence_cell'][int(i/2)*12+j]))
                    
            
            elif np.mod(i,2)==0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('<'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j]))   
                                                 
                else:
                    temp.append('$\pm$'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j])) 
                    
                                        
        writer.writerow({'0':temp[0],
                         '1':temp[1],
                         '2':temp[2],
                         '3':temp[3],
                         '4':temp[4],
                         '5':temp[5],
                         '6':temp[6],
                         '7':temp[7],
                         '8':temp[8],
                         '9':temp[9],
                         '10':temp[10],
                         '11':temp[11],})
        
data=pd.read_csv("result/occurrence_rate_2sigma_tot_cell_nofp_new.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,0.0013,0.00083,NaN,NaN,NaN,NaN,0.011,0.011,NaN,NaN,0.026
1,<0.0005,$\pm$0.00091,$\pm$0.00083,<0.0015,<0.0021,<0.003,<0.0043,$\pm$0.0075,$\pm$0.011,<0.013,<0.019,$\pm$0.026
2,0.0014,0.00051,0.0025,0.0046,0.0017,0.0012,0.018,0.012,NaN,NaN,NaN,NaN
3,$\pm$0.00098,$\pm$0.00051,$\pm$0.0014,$\pm$0.0027,$\pm$0.0017,$\pm$0.0012,$\pm$0.0091,$\pm$0.0082,<0.0097,<0.014,<0.021,<0.03
4,NaN,0.00095,0.0011,0.0025,0.0076,0.013,0.017,0.012,0.012,NaN,NaN,0.027
5,<0.00058,$\pm$0.00095,$\pm$0.0011,$\pm$0.0018,$\pm$0.0038,$\pm$0.0066,$\pm$0.0097,$\pm$0.0088,$\pm$0.012,<0.015,<0.022,$\pm$0.027
6,NaN,NaN,0.0032,0.017,0.015,0.0087,0.0034,0.018,0.045,0.054,0.076,0.03
7,<0.0006,<0.00086,$\pm$0.0019,$\pm$0.0055,$\pm$0.0059,$\pm$0.005,$\pm$0.0034,$\pm$0.01,$\pm$0.022,$\pm$0.027,$\pm$0.044,$\pm$0.03
8,NaN,NaN,0.0038,0.0092,0.016,0.014,0.02,0.0058,0.063,0.067,0.27,0.17
9,<0.00062,<0.00089,$\pm$0.0022,$\pm$0.0041,$\pm$0.0059,$\pm$0.0069,$\pm$0.01,$\pm$0.0058,$\pm$0.026,$\pm$0.033,$\pm$0.08,$\pm$0.075
